In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from ast import literal_eval
from langdetect import detect
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.externals import joblib

In [2]:
resumes = pd.read_csv('resumes_all_2_3.csv')

In [3]:
columns = ['job_titles', 'job_durations', 'job_descriptions', 'degrees', 'schools', 'graduation_dates']
for col in columns:
    resumes[col] = resumes[col].apply(lambda x: literal_eval(x)) # pandas read columns as string, change back to list

In [4]:
resumes.head()

,job_titles,job_durations,job_descriptions,degrees,schools,graduation_dates,job_type
0,"[Data Scientist, Business Inteligence Consultant]","[February 2013 to Present, January 2006 to Feb...","[2016 ~ Manager, Smart Solution Team in Inform...","[Master in Business Administration, Bachelor o...","[KAIST, Konkuk University]","[January 2012 to January 2013, March 1999 to F...",data+scientist
1,"[Director of Business Relations, Senior Data A...","[May 2018 to Present, May 2014 to July 2016]",[GIMME360 Data Scientist-Business Solution 2...,"[Master in applied mathematics, Bachelor in ap...","[University of Waterloo, University of Waterloo]","[January 2016 to January 2018, January 2010 to...",data+scientist
2,"[FREELANCE DEVELOPER, Data Scientist, DESIGN E...","[March 2016 to Present, April 2018 to August 2...","[Toronto, Canada• Completed 30+ freelancer Pro...",[BACHELORS in COMPUTER ENGINEERING],[UNIVERSITY OF TORONTO],[April 2021],data+scientist
3,"[Automation Test Engineer, Data Scientist, Dat...","[December 2018 to Present, August 2018 to Nove...",[Responsibilities: ⚫ GM Infotainment system...,[Master in Electrical and Computer Engineering...,"[University of Ottawa, Beijing Institute of Te...","[September 2015 to June 2017, September 2009 t...",data+scientist
4,"[Validation Scientist Co-Op (PEY), Data Collec...","[September 2017 to August 2018, May 2015 to Se...",[• Developing qualification document for a new...,[B.A.Sc in Electrical Engineering],[University of Toronto],[September 2014 to April 2019],data+scientist


In [5]:
resumes['resume_id'] = [i+1 for i in range(len(resumes))]
# have separate dfs for each column for easier data manipulation
titles = resumes[['resume_id', 'job_type', 'job_titles']]
durations = resumes[['resume_id', 'job_durations']]
descriptions = resumes[['resume_id', 'job_descriptions']]
degrees = resumes[['resume_id', 'job_type', 'degrees']]
schools = resumes[['resume_id', 'schools']]
graduation = resumes[['resume_id', 'graduation_dates']]

Make a dataframe for work experience

In [6]:
# split list into rows for job titles
titles = titles['job_titles'].apply(pd.Series) \
         .merge(titles, right_index=True, left_index=True) \
         .drop(['job_titles'], axis=1) \
         .melt(id_vars=['resume_id', 'job_type'], value_name="job_title")

In [7]:
# split list into rows for job durations
durations = durations['job_durations'].apply(pd.Series) \
         .merge(durations, right_index=True, left_index=True) \
         .drop(['job_durations'], axis=1) \
         .melt(id_vars=['resume_id'], value_name="job_duration")

In [8]:
# split list into rows for job descriptions
descriptions = descriptions['job_descriptions'].apply(pd.Series) \
         .merge(descriptions, right_index=True, left_index=True) \
         .drop(['job_descriptions'], axis=1) \
         .melt(id_vars=['resume_id'], value_name="job_description")

In [9]:
# merge all work exp variables
df_work = titles.merge(durations, on=['resume_id', 'variable']) \
           .merge(descriptions, on=['resume_id', 'variable']) \
           .dropna()
df_work.rename(columns={'variable':'job_id'}, inplace=True) # job_id: current job is 0 prev ones 1, 2...
df_work.sort_values(by=['resume_id', 'job_id'], inplace=True)
df_work = df_work.reset_index(drop=True)

In [10]:
# remove non english resumes
langdet = []
# lang detect sometimes cannot detect language for some text (e.g., empty string, too few characters)
for i in range(len(df_work)):                                         
    try:
        lang = detect(df_work.job_description[i])                                      
    except:                                                       
        lang = 'no'                                                  
    langdet.append(lang)

df_work['lang'] = langdet
df_work = df_work[df_work.lang == 'en']

In [11]:
df_work.reset_index(drop=True, inplace=True)
df_work.head()

,resume_id,job_type,job_id,job_title,job_duration,job_description,lang
0,1,data+scientist,0,Data Scientist,February 2013 to Present,"2016 ~ Manager, Smart Solution Team in Informa...",en
1,1,data+scientist,1,Business Inteligence Consultant,January 2006 to February 2013,2011 Social Media Analysis- Participate in the...,en
2,2,data+scientist,0,Director of Business Relations,May 2018 to Present,GIMME360 Data Scientist-Business Solution 20...,en
3,2,data+scientist,1,Senior Data Analyst-Customer Interaction Decis...,May 2014 to July 2016,SAS Fulltime• Translated business objectives ...,en
4,3,data+scientist,0,FREELANCE DEVELOPER,March 2016 to Present,"Toronto, Canada• Completed 30+ freelancer Proj...",en


In [12]:
len(df_work)

16377

In [13]:
# remove duplicates
df_work.drop_duplicates(['job_description'], keep='first', inplace=True)
df_work.reset_index(drop=True, inplace=True)

In [14]:
len(df_work)

14494

In [15]:
# save work df
df_work.to_csv('work_exp.csv', index=False)

make a dataframe for education

In [16]:
# split list into rows for degrees
degrees = degrees['degrees'].apply(pd.Series) \
         .merge(degrees, right_index=True, left_index=True) \
         .drop(['degrees'], axis=1) \
         .melt(id_vars=['resume_id', 'job_type'], value_name="degree")

In [17]:
# split list into rows for schools
schools = schools['schools'].apply(pd.Series) \
         .merge(schools, right_index=True, left_index=True) \
         .drop(['schools'], axis=1) \
         .melt(id_vars=['resume_id'], value_name="school")

In [18]:
# split list into rows for graduation dats
graduation = graduation['graduation_dates'].apply(pd.Series) \
             .merge(graduation, right_index=True, left_index=True) \
             .drop(['graduation_dates'], axis=1) \
             .melt(id_vars=['resume_id'], value_name="graduation")

In [19]:
# merge all education related variables
df_ed = degrees.merge(schools, on=['resume_id', 'variable']) \
               .merge(graduation, on=['resume_id', 'variable']) \
               .dropna()
df_ed.rename(columns={'variable':'education_id'}, inplace=True) # education_id: most recent ed is 0 prev ones 1, 2...
df_ed.sort_values(by=['resume_id'], inplace=True)
df_ed = df_ed.reset_index(drop=True)
df_ed = df_ed.drop(['job_type'], axis=1)

In [20]:
df_ed.head()

,resume_id,education_id,degree,school,graduation
0,1,0,Master in Business Administration,KAIST,January 2012 to January 2013
1,1,1,Bachelor of Engineering in Computer Engineering,Konkuk University,March 1999 to February 2004
2,2,0,Master in applied mathematics,University of Waterloo,January 2016 to January 2018
3,2,1,Bachelor in applied mathematics,University of Waterloo,January 2010 to January 2014
4,3,0,BACHELORS in COMPUTER ENGINEERING,UNIVERSITY OF TORONTO,April 2021


In [21]:
df_ed.to_csv('ed.csv', index=False)

merge work and ed dataframes

In [22]:
# join work exp and ed dataframes
resumes = df_work.join(df_ed.set_index('resume_id'), on='resume_id')
resumes.sort_values(by=['resume_id', 'job_id', 'education_id'], inplace=True)

In [23]:
for col in ['job_title', 'job_duration', 'job_description', 'degree', 'school', 'graduation']:
    resumes[col] = resumes[col].astype(str)

In [24]:
# split durations to start and end dates
resumes['job_duration'] = resumes.job_duration.apply(lambda x: x+' to ' if 'to' not in x else x)
resumes['job_start'] = resumes.job_duration.apply(lambda x: x.split(' to ')[0])
resumes['job_end'] = resumes.job_duration.apply(lambda x: x.split(' to ')[1])

resumes['graduation'] = resumes.graduation.apply(lambda x: ' to '+x if 'to' not in x else x)
resumes['school_start'] = resumes.graduation.apply(lambda x: x.split(' to ')[0])
resumes['school_end'] = resumes.graduation.apply(lambda x: x.split(' to ')[1] if len(x.split(' to '))>1 else '')

In [25]:
# change columns with dates to datetime object
columns = ['job_start', 'job_end', 'school_start', 'school_end']
for col in columns:
    resumes[col] = resumes[col].apply(lambda x: x.replace('Present', 'January 2019'))
    resumes[col] = pd.to_datetime(resumes[col])

# duration of each work experience (in months)
resumes['duration'] = round((resumes.job_end - resumes.job_start).dt.days/30)

In [26]:
# find highest education at start of each work exp
index_keep = resumes.job_start >= resumes.school_end # check if a degree is finished when a job starts
resumes = resumes.loc[index_keep].groupby(['resume_id', 'job_id']).apply(lambda x: x.iloc[0, :])
resumes.reset_index(drop=True, inplace=True) # remove multilevel index

# have columns for highest education and field of study
resumes['highest_ed'] = resumes.degree.apply(lambda x: x.split(' in ')[0])
resumes['field'] = resumes.degree.apply(lambda x: x.split(' in ')[1] if len(x.split(' in '))>1 else '')

In [27]:
resumes.head()

,resume_id,job_type,job_id,job_title,job_duration,job_description,lang,education_id,degree,school,graduation,job_start,job_end,school_start,school_end,duration,highest_ed,field
0,1,data+scientist,0,Data Scientist,February 2013 to Present,"2016 ~ Manager, Smart Solution Team in Informa...",en,0,Master in Business Administration,KAIST,January 2012 to January 2013,2013-02-01,2019-01-01,2012-01-01,2013-01-01,72.0,Master,Business Administration
1,1,data+scientist,1,Business Inteligence Consultant,January 2006 to February 2013,2011 Social Media Analysis- Participate in the...,en,1,Bachelor of Engineering in Computer Engineering,Konkuk University,March 1999 to February 2004,2006-01-01,2013-02-01,1999-03-01,2004-02-01,86.0,Bachelor of Engineering,Computer Engineering
2,2,data+scientist,0,Director of Business Relations,May 2018 to Present,GIMME360 Data Scientist-Business Solution 20...,en,0,Master in applied mathematics,University of Waterloo,January 2016 to January 2018,2018-05-01,2019-01-01,2016-01-01,2018-01-01,8.0,Master,applied mathematics
3,2,data+scientist,1,Senior Data Analyst-Customer Interaction Decis...,May 2014 to July 2016,SAS Fulltime• Translated business objectives ...,en,1,Bachelor in applied mathematics,University of Waterloo,January 2010 to January 2014,2014-05-01,2016-07-01,2010-01-01,2014-01-01,26.0,Bachelor,applied mathematics
4,4,data+scientist,0,Automation Test Engineer,December 2018 to Present,Responsibilities: ⚫ GM Infotainment system ...,en,0,Master in Electrical and Computer Engineering,University of Ottawa,September 2015 to June 2017,2018-12-01,2019-01-01,2015-09-01,2017-06-01,1.0,Master,Electrical and Computer Engineering


In [29]:
resumes.to_csv('resumes_merged.csv', index=False)